In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
import openai
import os
from datasets import Dataset 
from ragas.metrics import context_precision

from ragas import evaluate
from ragas.metrics import context_recall
import pandas as pd
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Document
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex

from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core import Settings
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
import pandas as pd
from llama_index.core.llama_pack import download_llama_pack
import pandas as pd




In [ ]:

queries = [
       "What are the steps required to set up a boilerplate project using the Emumba Plugin?",
        "Can you list some of the key features provided by the Emumba Plugin for React applications?",
        "What is the purpose of the generateFiles function in the project setup generator, and how does it use the options provided?",
        "Describe the role of addDependenciesToPackageJson in the project setup process.",
        "How does the ProjectSetupGeneratorSchema interface influence the behavior of the project setup generator?",
        "Explain how the project configuration is added to the workspace using addProjectConfiguration in the context of the setup process.",
        "Describe the process and the purpose of creating a test project in the beforeAll setup of the emumba-plugin tests.",
        "How does the test for emumba-plugin ensure that the plugin is properly installed and functional within a generated project?",
        "there's a function used to create a test project directory. Output the code snippet that showcases how this directory is created and initialized.",
         '''Given the following incomplete snippet, complete the function to add a specific dependency to the project's package.json. 
           Assume the function addDependenciesToPackageJson is already imported.
            function enhancePackageJson(tree: Tree, projectName: string) {
            // Add 'react-redux' as a dependency
            addDependenciesToPackageJson(tree, projectName, {
                'react-redux': '^7.2.0'
            }, {});
            // Complete the function to also add 'redux' as a dependency
        }'''
]





In [ ]:
os.environ["OPENAI_API_KEY"]  = "your-key"


### Recursive Retriver

In [ ]:
documents = SimpleDirectoryReader('data/Boilerplate').load_data()
doc_nodes = Settings.node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(doc_nodes )
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    verbose=True,
)

response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever
)

In [ ]:
def recursiveretriver(queries,query_engine):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        ground_truth = df.iloc[i]['ground truth']  
        response = query_engine.query(query)
        result.append({'question': query, 'answer': str(response), 'ground_truths': ground_truth})
    return result


In [ ]:
result=recursiveretriver(queries,query_engine)

### Automerging retriver

In [ ]:
query_str = [
    ("What are the steps required to set up a boilerplate project" "using the Emumba Plugin?"),
    ("Can you list some of the key features provided"  "by the Emumba Plugin for React applications?"),
    ("What is the purpose of the generateFiles function in the project setup generator," "and how does it use the options provided?"),
    ("Describe the role of addDependenciesToPackageJson" "in the project setup process."),
    ("How does the ProjectSetupGeneratorSchema interface influence the behavior" "of the project setup generator?"),
    ("Explain how the project configuration is added to the workspace" "using addProjectConfiguration in the context of the setup process."),
    ("Describe the process and the purpose of creating a test project"  "in the beforeAll setup of the emumba-plugin tests."),
    ("How does the test for emumba-plugin ensure that the plugin is properly installed" "and functional within a generated project?"),
    ("there's a function used to create a test project directory." "Output the code snippet that showcases how this directory is created and initialized."),
    ("Given the following incomplete snippet, complete the function to add a specific dependency to the project's package.json."
     "Assume the function addDependenciesToPackageJson is already imported.\nfunction enhancePackageJson(tree: Tree, projectName: string) {\n    // Add 'react-redux' as a dependency\n    addDependenciesToPackageJson(tree, projectName, {\n        'react-redux': '^7.2.0'\n    }, {});\n    // Complete the function to also add 'redux' as a dependency\n}")
]


In [ ]:

doc_text = "\n\n".join([d.get_content() for d in documents])
docs = [Document(text=doc_text)]
node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)
leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)
base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)





In [ ]:
def automergingretrival(query_str,retriever):
    result=[]
    file_path = 'negative_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(query_str, start=0): 
        ground_truth = df.iloc[i]['ground truth']  
        nodes = retriever.retrieve(query)
        query_engine = RetrieverQueryEngine.from_args(retriever)
        response = query_engine.query(query)
       
        context=[]
        for node_with_score in nodes:
            text_content = node_with_score.node.text
            context.append(text_content)
        

        
        result.append({'question': query, 'answer': str(response), 'ground_truths': ground_truth,'contexts': context})
    return result
    

In [ ]:
result=automergingretrival(queries,base_retriever)

## Sentence window Retriever Pack

In [ ]:

SentenceWindowRetrieverPack = download_llama_pack(
    "SentenceWindowRetrieverPack",
    "./sentence_window_retriever_pack",
)
sentence_window_retriever_pack = SentenceWindowRetrieverPack(
    documents,
)

In [ ]:
def sentencewindowretrieval(sentence_window_retriever_pack,queries):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        ground_truth = df.iloc[i]['ground truth']  
        response = sentence_window_retriever_pack.run(query)
        context=[]
        for node_with_score in response.source_nodes:
            node = node_with_score.node  
            metadata = node.metadata  
            if 'window' in metadata:
                window_content = metadata['window']
                context.append(window_content)
            else:
                context.append(None)
        result.append({'question': query, 'answer': str(response), 'ground_truths': ground_truth,'contexts': context})
    return result

In [ ]:
result=sentencewindowretrieval(sentence_window_retriever_pack,queries)